## Optimaze the Linkedin parsing code 

In [114]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import time
import csv
import pandas as pd

In [115]:
## First we need a browser to run the selenium automation 
browser = webdriver.Chrome()

In [116]:
## Next let's Open the page that we will scrape in our case we will scrape the linkedin platform 
# Following the step we are in the sign in page of linkedin and it's time to insert our personal informations
browser.get('https://www.linkedin.com/login')

In [117]:
## In this step I will read a file that contains my informations in order to keep them private
file = open("info.txt")
lines = file.readlines()
username = lines[0]
password = lines[1]

In [118]:
## Let's Sign in
# Find the username box and add your username
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

In [119]:
# Find the password box and add your password
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

In [120]:
## click to submit join option
elementID.submit()

In [121]:
## Determine the link of the search  
link = 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL'

In [122]:
## Get the link 
browser.get(link)
time.sleep(2)

In [123]:
## Now we are in the page that we want to scrape
## I will use the beautiful soup library to scrape the page
# Our goal is to parse all the companies with some details that will be useful for the competition analysis 
page = bs(browser.page_source, "html.parser")

In [124]:
## Creating a function in order to collect the information that we need from the pages

def get_features(card):
    href = card.a
    title = card.find("span", "entity-result__title-line flex-shrink-1 entity-result__title-text--black").text.replace("\n","")
    link = href.get("href")
    try:
        sector = card.find("div", "entity-result__primary-subtitle t-14 t-black").text.replace("\n", "")
    except:
        sector = "None"
    followers = card.find("div", "entity-result__secondary-subtitle t-14").text.replace("\n","")
    try:
        description = card.find("p", "entity-result__summary t-12 t-black--light mb1").text.replace("\n","")
    except:
        description = "None"
    try:  
        current_positions = card.find("span", "entity-result__simple-insight-text").text.replace("\n","")
    except:
        current_positions = "None"
        
    feature = (title, link, sector, followers, description, current_positions)    
    return feature

In [127]:
## Determining the number of the pages that you want to parse data from.
n = input("Determing the number of the pages that you want to scrape:")

Determing the number of the pages that you want to scrape:4


In [132]:
## Append the different url's into a list
pages = []
pages.append(link) 

def number_of_pages(n):
    new_url = 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL' +'&page={num:.2f}'
    for i in range(2,n+1):
        try:
            pages.append(new_url.format(num=i))
        except AttributeError:
            break
    return pages

In [133]:
## Inspecting the URLs list
n = int(n)
number_of_pages(n)

['https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=2.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=3.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=4.00']

In [134]:
## Iterating through the pages and parse all the features used the scraping function
final_list = []
for link in pages:
    browser.get(link)
    page = bs(browser.page_source, "html.parser")
    cards = page.find_all("div", "entity-result")
    
    time.sleep(4)
    
    for card in cards:
            features = get_features(card)
            final_list.append(features)
    time.sleep(2)

In [135]:
## Print the length  of the link
len(final_list)

40

In [136]:
## Convert the list to csv file
import csv
with open('new_oper.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["title", "link", "sector", "followers", "description", "current_positions"])
        writer.writerows(final_list)

In [137]:
df = pd.read_csv("new_oper.csv")
df.shape

(40, 6)

In [88]:
## Remove duplicates 
new_df = df.drop_duplicates(subset=['title'])
new_df.shape

(50, 6)

In [89]:
## Save the dataframe to a new csv file 
new_df.to_csv("final_file.csv", index=False)